In [ ]:
def compound_moa_classes(clue_sig, compounds1_2, moas, cell_name_string, cell_name, lower_doses, higher_doses, head_num = 5):
    '''Creates a table that assesses the number of transcriptomic profiles given a certain dosage range and a selection of 
    cell line from the clue.io database
    
    Input:
    clue_sig:         metadata from all transcriptomic level 5 signatures
    compounds1_2:     combined list of compounds found in SPECS V1 and V2.
    moas:             list of strings representing a subset of MoA classes to investigatedd
    cell_name_string: list of strings representing cell line name
    cell_name:        list of strings of the cell lines to include
    lower_doses:      list of integers representing lower range of dosages
    higher_doses:     list of integers representing higher range of dosages
    head_num:         int that determines number of rows displayed by datafram
    
    Output:
    Several data frames of a length determined by head_num
    '''
    
    for i, s in zip(cell_name, cell_name_string):
        for a,f in zip(lower_doses, higher_doses):
            clue_sig = clue_sig[clue_sig.pert_id.isin(compounds1_2.CUSTOMER_ID)]
            check = clue_sig[clue_sig.cell_iname.isin(i)]
            check_with_correct_dosage = check[check.pert_dose.between(a,f)].reset_index(drop=True)
            unique_all = check_with_correct_dosage.drop_duplicates(subset=["pert_id"])
            
            # printing MoA classes with x number of compounds targeting specific MoA class 
            print(f" \033[1m{'Cell Line(s):' : >20}\033[0m  {s}")
            print(f" \033[1m{'Dosage Range:' : >20}\033[0m  [{a},{f}]")
            print(f" \033[1m{'All Compounds Found in SPECS v1 and V2' : >20}\033[0m")
            print(unique_all.moa.value_counts().head(head_num))
            print(f'Enantiomers: {unique_all[unique_all.duplicated("Compound ID")].shape[0]}')
            chosen_MoA_classes = check_with_correct_dosage[check_with_correct_dosage.moa.isin(moas)]
            unique_chosen_MoA_classes = chosen_MoA_classes.drop_duplicates(subset=["pert_id"])
            
            # printing MoA classes with x number of compounds targeting specific MoA class (Chosen Subset)
            print(f" \033[1m{'Selected MoAS' : >20}\033[0m" )
            print(unique_chosen_MoA_classes.moa.value_counts().head(head_num))
            print(f'Enantiomers: {unique_chosen_MoA_classes[unique_chosen_MoA_classes.duplicated("Compound ID")].shape[0]}')
            print(chr(10))
